In [1]:
# Lo primero que hacemos es importar todas las librerias necesarias.
#importamos librerias para tratamiento de datos
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format
import re
import math as mat

#importamos librerias la visualizacion de datos
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [6]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../../dataset/properati.csv"
data = pd.read_csv(filename)

display(data.sample(n=6))
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
65631,65631,sell,store,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,"3,860,255.0",NaN,nan,...,89.0,nan,"28,089.887640449437",nan,nan,nan,http://www.properati.com.ar/1acmt_venta_local_...,Te invitamos a conocer:FECHA DE ENTREGA: JUNIO...,*** Invertí local en Nva Cba***,https://thumbs4.properati.com/9/XZs1FxEnWHAtIP...
84561,84561,sell,house,La Lucila,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,"3,432,310.0",NaN,nan,...,800.0,897.4358974358977,"1,750.0",nan,nan,nan,http://www.properati.com.ar/1b1uy_venta_casa_l...,Imponente Residencia de estilo Francés a valor...,Imponente Residencia de estilo Frances en La L...,https://thumbs4.properati.com/6/8ZtV-emU-bANda...
119088,119088,sell,apartment,Villa Ballester,|Argentina|Bs.As. G.B.A. Zona Norte|General Sa...,Argentina,Bs.As. G.B.A. Zona Norte,"3,427,467.0",NaN,nan,...,40.0,nan,"1,775.0",nan,nan,nan,http://www.properati.com.ar/1cg6d_venta_depart...,"Departamento a estrenar, con excelentes materi...",Departamento - Villa Ballester,https://thumbs4.properati.com/1/jL-fppEJEZClPo...
61104,61104,sell,apartment,Villa Luro,|Argentina|Capital Federal|Villa Luro|,Argentina,Capital Federal,"3,427,407.0","-34.6385616,-58.5069507",-34.6385616,...,38.0,"2,023.8095238095239","2,236.842105263158",nan,2.0,"1,700.0",http://www.properati.com.ar/1a4v3_venta_depart...,Venta de Departamento 2 AMBIENTES en Villa Lur...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/6/bsWpP9odZkxIrs...
119723,119723,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.0","-34.555441,-58.450287",-34.555440999999995,...,59.0,nan,"2,694.915254237288",nan,3.0,nan,http://www.properati.com.ar/1ch0u_venta_depart...,Corredor Responsable: MARIA NATALE - CUCICBA 6...,Olazabal 1500 -VENTA 3 amb al frente muy luminoso,https://thumbs4.properati.com/1/2tfFuEKmhpbK7F...
49411,49411,sell,apartment,Nuñez,|Argentina|Capital Federal|Nuñez|,Argentina,Capital Federal,"3,430,348.0","-34.556256,-58.468448",-34.556256,...,nan,nan,nan,nan,4.0,nan,http://www.properati.com.ar/19c4f_venta_depart...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Núñez,https://thumbs4.properati.com/7/aixo-AyNrJaqR_...


(Cantidad de Filas, Cantidad de columnas) -> (121220, 26)


In [ ]:
print (data.isna().sum()/data.shape[0]*100)

In [ ]:
# Columna surface_total_in_m2: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.surface_total_in_m2.isnull().sum()}')
print(f'Porcentaje de registros con NaN: {data.surface_total_in_m2.isnull().mean()*100}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['surface_total_in_m2']).size().reset_index(name='count'))

In [ ]:
#graficamos el dato por tipo de propiedad con el fin de observar los rangos de areas de superficie total que incluye areas adicionales a la superficie cubierta.
#En esta grafica podemos observar la variación en la media de las areas que existen de acuerdo al Estado.
data.groupby('state_name')['surface_total_in_m2'].mean().plot.bar()

# Detalle de análisis de columnas:
* `surface_total_in_m2 ` -> Esta columna tiene 19.907 datos nulos o su equivalente 16,42%. El precio se calcula sobre la superficie total asi que debemos revisar la relevancia de esta variable.

In [ ]:
data.surface_total_in_m2.describe()

### Detalle de valores extremos en la  columna `surface_total_in_m2`:

En el catplot elaborado para esta variable comparada por tipo de propiedad podemos observar como en apartamentos se evidencias valores muy por encima de los valores medios, en el caso de las casas se observan algunas propiedades alejadas de la media.

In [ ]:
sns.catplot(data=data, x= 'surface_total_in_m2', y="property_type", kind="box", height=5, aspect=5,)


### Regex para rellenar la columna `surface_total_in_m2`:

Vamos a extraer datos de las columnas title y description para extraer esta información

In [7]:
# Buscamos superficies en metros cuadrado en los titulos y en la descripción

#el area más pequeña es de 0 m2 hay 383 registos en ceros, trae a la base m2, debemos eliminarlos y transformar . a , ???

data[['description', 'title']] = data[['description', 'title']].apply(lambda x: x.astype(str))

patron = r'([0-9]\.?\,?[0-9])\s*[metros|METROS|m|M|mts|MTS]\.*\s*2'


regex = re.compile(patron, flags = re.IGNORECASE)

superficie_total_titulo=data['title'].replace(np.NAN,"").apply(lambda x: regex.findall(x)[0] if regex.findall(x) else np.NaN)
data['surface_total_in_m2'].fillna(superficie_total_titulo, inplace=True)

superficie_total_descripcion=data['description'].replace(np.NAN,"").apply(lambda x: regex.findall(x)[0] if regex.findall(x) else np.NaN)
data['surface_total_in_m2'].fillna(superficie_total_descripcion, inplace=True)

print (data.isna().sum()/data.shape[0]*100)
print (f'entradas: {data.shape[0]}')

Unnamed: 0                                    0.0
operation                                     0.0
property_type                                 0.0
place_name                   0.018973766705164163
place_with_parent_names                       0.0
country_name                                  0.0
state_name                                    0.0
geonames_id                    15.440521366111204
lat-lon                         42.52598581092229
lat                             42.52598581092229
lon                             42.52598581092229
price                          16.837155584886982
currency                        16.83798053126547
price_aprox_local_currency     16.837155584886982
price_aprox_usd                16.837155584886982
surface_total_in_m2             25.83814552054117
surface_covered_in_m2          16.422207556508827
price_usd_per_m2               43.394654347467416
price_per_m2                   27.686850354726943
floor                           93.48374855634384


In [28]:
data.surface_total_in_m2.sample(50)

101764      39.0
71289       75.0
85496       84.0
86441       47.0
40795        NaN
78610       50.0
116438       NaN
88164       72.0
99764       68.0
69537         84
32216         19
52500         69
27133       48.0
44802        NaN
100404      46.0
35296       99.0
59305       80.0
76612        NaN
94999      300.0
69239       49.0
38214        NaN
26304       35.0
104507      33.0
3853       450.0
88534      156.0
81962       45.0
77865       55.0
112867     110.0
85179        NaN
89246      200.0
23955      660.0
20728      200.0
43486      120.0
49761       37.0
102241      48.0
89134       82.0
2589       224.0
36008    1,055.0
21920         36
86031       30.0
73805      246.0
109631       NaN
55708       55.0
9948        43.0
35082        NaN
48126       53.0
111766     316.0
17009        NaN
96628        NaN
8027         NaN
Name: surface_total_in_m2, dtype: object